In [1]:

from typing import Any, Dict

import tensorflow as tf
import keras
from keras import backend as K
from keras.constraints import max_norm
from keras.layers import (
    Activation,
    Add,
    AveragePooling2D,
    AvgPool2D,
    BatchNormalization,
    Concatenate,
    Conv2D,
    Dense,
    DepthwiseConv2D,
    Dropout,
    Flatten,
    Input,
    Lambda,
    LayerNormalization,
    MaxPooling2D,
    Permute,
)
from keras.models import Model, Sequential
from scikeras.wrappers import KerasClassifier

from moabb.pipelines.utils_deep_model import EEGNet, EEGNet_TC

from pickle import load

import keras
# from braindecode import EEGClassifier
# from braindecode.models import EEGInception
# from scikeras.wrappers import KerasClassifier
# from sklearn.pipeline import Pipeline, make_pipeline
# from skorch.callbacks import EarlyStopping, EpochScoring
# from skorch.dataset import ValidSplit

from moabb import set_log_level
from moabb.pipelines.features import StandardScaler_Epoch
from moabb.utils import setup_seed

# Datasets
from custom_datasets.fatigue_mi import FatigueMI
from custom_datasets.norm_cho import NormCho2017
from moabb.datasets import Cho2017

# Moabb evaluation methods
from moabb.evaluations import CrossSessionEvaluation, WithinSessionEvaluation

# Moabb paradigms
from moabb.paradigms import LeftRightImagery

# Scikitlearn imports
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from moabb.pipelines.features import Resampler_Epoch, Convert_Epoch_Array, StandardScaler_Epoch


%load_ext autoreload
%autoreload 3

2024-02-15 17:36:22.756183: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-15 17:36:23.515414: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-15 17:36:23.517413: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-15 17:36:24.752715: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_types is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_channels_r

To use the get_shape_from_baseconcar, InputShapeSetterEEG, BraindecodeDatasetLoaderyou need to install `braindecode`.`pip install braindecode` or Please refer to `https://braindecode.org`.


In [2]:
# Set up reproducibility of Tensorflow
setup_seed(42)

We try to set the torch seeds, but it seems that torch is not installed. Please refer to `https://pytorch.org/` to install if you need to use this deep learning module.


False

In [26]:
# =================================================================================
# DeepConvNet
# =================================================================================
class KerasDeepConvNet(KerasClassifier):
    """Keras implementation of the Deep Convolutional Network as described in
    [1]_.

    This implementation is taken from code by the Army Research Laboratory (ARL)
    at https://github.com/vlawhern/arl-eegmodels

    We use the original parameter implemented on the paper.

    Note that this implementation has not been verified by the original
    authors.

    References
    ----------
    .. [1] Schirrmeister, R. T., Springenberg, J. T., Fiederer, L. D. J., Glasstetter, M., Eggensperger,
           K., Tangermann, M., ... & Ball, T. (2017). Deep learning with convolutional neural networks
           for EEG decoding and visualization. Human brain mapping, 38(11), 5391-5420.
           https://doi.org/10.1002/hbm.23730

    Notes
    -----
    .. versionadded:: 0.5.0
    """

    def __init__(
        self,
        loss,
        optimizer="Adam",
        epochs=1000,
        batch_size=64,
        verbose=0,
        random_state=None,
        validation_split=0.2,
        history_plot=False,
        path=None,
        **kwargs,
    ):
        super().__init__(**kwargs)

        self.loss = loss
        if optimizer == "Adam":
            optimizer = tf.keras.optimizers.Adam(learning_rate=0.0009)

        self.optimizer = optimizer
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose
        self.random_state = random_state
        self.validation_split = validation_split
        self.history_plot = history_plot
        self.path = path

    def _keras_build_fn(self, compile_kwargs: Dict[str, Any]):
        input_main = Input(shape=(self.X_shape_[1], self.X_shape_[2], 1))

        block1 = Conv2D(
            25,
            (1, 10),
            input_shape=(self.X_shape_[1], self.X_shape_[2], 1),
            kernel_constraint=max_norm(2.0, axis=(0, 1, 2)),
        )(input_main)
        block1 = Conv2D(
            25,
            (self.X_shape_[1], 1),
            kernel_constraint=max_norm(2.0, axis=(0, 1, 2)),
        )(block1)
        block1 = BatchNormalization(epsilon=1e-05, momentum=0.9)(block1)
        block1 = Activation("elu")(block1)
        block1 = MaxPooling2D(pool_size=(1, 3), strides=(1, 3))(block1)
        block1 = Dropout(0.5)(block1)

        block2 = Conv2D(50, (1, 10), kernel_constraint=max_norm(2.0, axis=(0, 1, 2)))(
            block1
        )
        block2 = BatchNormalization(epsilon=1e-05, momentum=0.9)(block2)
        block2 = Activation("elu")(block2)
        block2 = MaxPooling2D(pool_size=(1, 3), strides=(1, 3))(block2)
        block2 = Dropout(0.5)(block2)

        block3 = Conv2D(100, (1, 10), kernel_constraint=max_norm(2.0, axis=(0, 1, 2)))(
            block2
        )
        block3 = BatchNormalization(epsilon=1e-05, momentum=0.9)(block3)
        block3 = Activation("elu")(block3)
        block3 = MaxPooling2D(pool_size=(1, 3), strides=(1, 3))(block3)
        block3 = Dropout(0.5)(block3)

        block4 = Conv2D(200, (1, 10), kernel_constraint=max_norm(2.0, axis=(0, 1, 2)))(
            block3
        )
        block4 = BatchNormalization(epsilon=1e-05, momentum=0.9)(block4)
        block4 = Activation("elu")(block4)
        block4 = MaxPooling2D(pool_size=(1, 3), strides=(1, 3))(block4)
        block4 = Dropout(0.5)(block4)

        flatten = Flatten()(block4)

        dense = Dense(self.n_classes_, kernel_constraint=max_norm(0.5))(flatten)
        softmax = Activation("softmax")(dense)

        model = Model(inputs=input_main, outputs=softmax)

        model.compile(loss=compile_kwargs["loss"], optimizer=compile_kwargs["optimizer"])

        ## Pruning

        # prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

        # pruning_params = {
        #     'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(0.5, begin_step=0, frequency=100)
        # }

        # callbacks = [
        # tfmot.sparsity.keras.UpdatePruningStep()
        # ]

        # pruned_model = prune_low_magnitude(model, **pruning_params)

        # # Use smaller learning rate for fine-tuning
        # opt = keras.optimizers.Adam(learning_rate=1e-5)

        # pruned_model.compile(
        # loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        # optimizer=opt,
        # metrics=['accuracy'])

        return model
        

keras_deep_conv_net = KerasDeepConvNet(
    loss="sparse_categorical_crossentropy",
    optimizer="Adam",
    epochs=25,
    batch_size=64,
    verbose=1,
    random_state=None,
    validation_split=0.2,
    history_plot=False,
    path=None,
)
# keras_deep_conv_net.save("keras_deep_conv_net.h5")

In [27]:
# model_keras_pruned = keras.models.load_model("keras_deep_conv_net.h5")
model_keras_pruned = keras_deep_conv_net
# Now we need to instantiate a new SciKeras object since we only saved the Keras model
resampler_epoch = Resampler_Epoch(sfreq=250)
convert_epoch_array = Convert_Epoch_Array()
standardscaler_epoch = StandardScaler_Epoch()
Keras_DeepConvNet_Trained_Pruned = make_pipeline(
    # resampler_epoch,
    # convert_epoch_array,
    # standardscaler_epoch,
    model_keras_pruned
)

Keras_DeepConvNet_Trained_Pruned

Pipeline(steps=[('kerasdeepconvnet',
                 KerasDeepConvNet(build_fn=None, callbacks=None, class_weight=None, epochs=25, loss='sparse_categorical_crossentropy', metrics=None, model=None, optimizer=<keras.optimizers.adam.Adam object at 0x7f2ce83ff650>, run_eagerly=False, shuffle=True, validation_batch_size=None, verbose=1, warm_start=False))])

In [29]:
# # model_keras_pruned = keras.models.load_model("keras_deep_conv_net.h5")
# model_keras_pruned = keras_deep_conv_net
# # Now we need to instantiate a new SciKeras object since we only saved the Keras model
# Keras_DeepConvNet_Trained_Pruned = KerasClassifier(build_fn = model_keras_pruned._keras_build_fn)

# Restrict this example only on the first two subject of BNCI2014_001
fat_dataset = FatigueMI()
norm_cho_dataset = NormCho2017()
cho_dataset = Cho2017()

fat_dataset.subject_list = fat_dataset.subject_list[:1]
norm_cho_dataset.subject_list = norm_cho_dataset.subject_list[:32]
cho_dataset.subject_list = cho_dataset.subject_list[:32]

datasets = [
    fat_dataset,
    # norm_cho_dataset,
    # cho_dataset
]

evaluation_dict = {
    "WithinSession": WithinSessionEvaluation
}
paradim_dict = {
    "LeftRightImagery": LeftRightImagery
}
pipeline_dict = {
    "Keras_DeepConvNet_Trained_Pruned": Pipeline(
        [
            # ("StandardScaler_Epoch", StandardScaler_Epoch),
            ("Keras_DeepConvNet_Trained", Keras_DeepConvNet_Trained_Pruned),
        ]
    ),
}

results_dict = {}

for model_text in pipeline_dict.keys():
    for evaluation_text in ["WithinSession"]: #, "CrossSession", "CrossSubject"]:
        for paradim_text in ["LeftRightImagery"]: #, "FilterBankMotorImagery"]:
            evaluation = evaluation_dict[evaluation_text](
                paradigm=paradim_dict[paradim_text](), datasets=datasets, overwrite=True
            )
            results = evaluation.process({ model_text: pipeline_dict[model_text] })
            results_dict[model_text + "_" + evaluation_text + "_" + paradim_text] = results



FatigueMI-WithinSession:   0%|          | 0/1 [00:00<?, ?it/s]

No hdf5_path provided, models will not be saved.
Epoch 1/25


/home/araz/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_function_transformer.py:310: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  return func(X, **(kw_args if kw_args else {}))
2024-02-15 18:01:48.098237: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [68,1]
	 [[{{node Placeholder/_11}}]]
2024-02-15 18:01:48.098391: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [68,1]
	 [[{{node Placeholder/_11}}]]


2/2 [==============================] - 1s 136ms/step - loss: 0.8355 - val_loss: 0.7491
Epoch 2/25


2024-02-15 18:01:49.365689: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [18,1]
	 [[{{node Placeholder/_11}}]]
2024-02-15 18:01:49.365838: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [18,1]
	 [[{{node Placeholder/_11}}]]


...

2024-02-15 18:02:11.404499: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [21,20,601]
	 [[{{node Placeholder/_10}}]]
2024-02-15 18:02:11.404663: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [21,20,601]
	 [[{{node Placeholder/_10}}]]
2024-02-15 18:02:11.559375: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and

2/2 [==============================] - 1s 140ms/step - loss: 1.8173 - val_loss: 0.7626
Epoch 2/25


2024-02-15 18:02:12.832956: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [18,1]
	 [[{{node Placeholder/_11}}]]
2024-02-15 18:02:12.833111: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [18,1]
	 [[{{node Placeholder/_11}}]]


2/2 [==============================] - 0s 33ms/step - loss: 0.8064 - val_loss: 0.6912
Epoch 3/25
2/2 [==============================] - 0s 32ms/step - loss: 0.7462 - val_loss: 0.6831
Epoch 4/25
2/2 [==============================] - 0s 34ms/step - loss: 0.9176 - val_loss: 0.6823
Epoch 5/25
2/2 [==============================] - 0s 42ms/step - loss: 0.9231 - val_loss: 0.6783
Epoch 6/25
2/2 [==============================] - 0s 31ms/step - loss: 0.8680 - val_loss: 0.6714
Epoch 7/25
2/2 [==============================] - 0s 31ms/step - loss: 0.7894 - val_loss: 0.6675
Epoch 8/25
2/2 [==============================] - 0s 33ms/step - loss: 0.7631 - val_loss: 0.6658
Epoch 9/25
2/2 [==============================] - 0s 31ms/step - loss: 0.7342 - val_loss: 0.6736
Epoch 10/25
2/2 [==============================] - 0s 29ms/step - loss: 0.7751 - val_loss: 0.6924
Epoch 11/25
2/2 [==============================] - 0s 31ms/step - loss: 0.7571 - val_loss: 0.7178
Epoch 12/25
2/2 [======================

2024-02-15 18:02:17.266781: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [21,20,601]
	 [[{{node Placeholder/_10}}]]
2024-02-15 18:02:17.266925: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [21,20,601]
	 [[{{node Placeholder/_10}}]]
FatigueMI-WithinSession: 100%|██████████| 1/1 [00:29<00:00, 29.97s/it]


In [33]:
from rich.pretty import pprint as rpprint

rpprint(results_dict['Keras_DeepConvNet_Trained_Pruned_WithinSession_LeftRightImagery'])

│     score      time  samples subject session  channels  n_sessions  \
0  0.555537  5.871249    108.0       1       0        20           1   
│    
│    dataset                          pipeline  
0  FatigueMI  Keras_DeepConvNet_Trained_Pruned